In [1]:
import random

In [5]:
col_count = 10
row_count = 20

# discretized = [random.randint(0, 1) for _ in range(col_count)]
discretized = [1 for _ in range(col_count)]

with open("dummy_discretization_flags.csv", "w") as file:
    file.write('\t'.join(str(flag) for flag in discretized) + '\n')

dofs = []
for flag in discretized:
    if flag:
        dof = random.randint(2,20)
    else:
        dof = 0
    dofs.append(dof)
    
print(dofs)
        
with open("dummy_X.csv", "w") as file:
    for _ in range(row_count):
        row = []
        for flag, dof in zip(discretized, dofs):
            if flag:
                row.append(random.randint(0, dof - 1))
            else:
                row.append(random.random())
        file.write('\t'.join(str(num) for num in row) + '\n')
        
with open("dummy_y.csv", "w") as file:
    for _ in range(row_count):
        file.write(str(random.randint(0,2)) + '\n')

[3, 13, 5, 14, 5, 9, 4, 3, 4, 10]


---

In [4]:
import random
import numpy as np

In [5]:
total_col_count = 1245
total_row_count = 100

fat_dof = 5
lean_A_dof = 6
lean_B_dof = 7
lean_C_dof = 8

fat_discretized = False
lean_A_discretized = True
lean_B_discretized = True
lean_C_discretized = True

reg_fat_count = 222
reg_lean_A_count = 13
reg_lean_B_count = 12
reg_lean_C_count = 11

heur_fat_count = 542
heur_lean_A_count = 42
heur_lean_B_count = 34
heur_lean_C_count = 0

contr_fat_count = 200
contr_lean_A_count = 124
contr_lean_B_count = 0
contr_lean_C_count = 45

assert reg_fat_count + heur_fat_count + contr_fat_count +\
reg_lean_A_count + heur_lean_A_count + contr_lean_A_count +\
reg_lean_B_count + heur_lean_B_count + contr_lean_B_count +\
reg_lean_C_count + heur_lean_C_count + contr_lean_C_count \
== total_col_count

In [6]:
column_range_counts = \
[reg_fat_count,
 reg_lean_A_count,
 reg_lean_B_count,
 reg_lean_C_count,
 heur_fat_count,
 heur_lean_A_count,
 heur_lean_B_count,
 heur_lean_C_count,
 contr_fat_count,
 contr_lean_A_count,
 contr_lean_B_count,
 contr_lean_C_count
]

range_discretized = \
[fat_discretized,
 lean_A_discretized,
 lean_B_discretized,
 lean_C_discretized
] * 3

range_dofs = \
[fat_dof,
 lean_A_dof,
 lean_B_dof,
 lean_C_dof    
] * 3

range_bounds = np.cumsum(column_range_counts).tolist()
range_bounds

[222, 235, 247, 258, 800, 842, 876, 876, 1076, 1200, 1200, 1245]

In [7]:
for range_bound_1, range_bound_2, range_dof, range_dicretized_status\
in zip([0] + range_bounds[:-1], range_bounds, range_dofs, range_discretized):
    print(range_bound_1, range_bound_2, range_dof, range_dicretized_status)

0 222 5 False
222 235 6 True
235 247 7 True
247 258 8 True
258 800 5 False
800 842 6 True
842 876 7 True
876 876 8 True
876 1076 5 False
1076 1200 6 True
1200 1200 7 True
1200 1245 8 True


In [8]:
fat_tile_width = 100

tile_bounds = [0]
tile_group_bounds = [0]
tile_discretized = []
tile_dofs = []
in_fat_tile_group = True

for range_bound_1, range_bound_2, range_dof, range_dicretized_status\
in zip([0] + range_bounds[:-1], range_bounds, range_dofs, range_discretized):
    
    if not range_dicretized_status:
        if not in_fat_tile_group:
            tile_group_bounds.append(len(tile_bounds) - 1)
        in_fat_tile_group = True
        number_of_tiles = (range_bound_2 - range_bound_1) // fat_tile_width
        for _ in range(number_of_tiles):
            tile_bounds.append(tile_bounds[-1] + fat_tile_width)
            tile_discretized.append(False)
            tile_dofs.append(range_dof)
        tile_group_bounds.append(len(tile_bounds) - 1)
        width_of_leftover_margin = (range_bound_2 - range_bound_1) % fat_tile_width
        if width_of_leftover_margin:
            tile_bounds.append(tile_bounds[-1] + width_of_leftover_margin)
            tile_discretized.append(False)
            tile_dofs.append(range_dof)
    else:
        in_fat_tile_group = False
        tile_bounds.append(tile_bounds[-1] + (range_bound_2 - range_bound_1))
        tile_discretized.append(True)
        tile_dofs.append(range_dof)

if not in_fat_tile_group:
    tile_group_bounds.append(len(tile_bounds) - 1)

In [9]:
tile_group_bounds

[0, 2, 6, 11, 15, 17, 20]

In [12]:
X = np.zeros((total_row_count, total_col_count))

for region_index, (tile_bound_1, tile_bound_2, tile_discretized_status, dof)\
in enumerate(zip(tile_bounds[:-1],tile_bounds[1:], tile_discretized, tile_dofs)):
    if region_index in tile_group_bounds:
        str_ = "* "
    else:
        str_ = "  "
    str_ += f"{region_index:>2} {tile_bound_1:>4} {tile_bound_2:>4} {tile_discretized_status} {dof}"
    print(str_)
    for column_index in range(tile_bound_1, tile_bound_2):
        if not tile_discretized_status:
            for row_index in range(total_row_count):
                X[row_index, column_index] = random.gauss(0, 1)
        else:
            for row_index in range(total_row_count):
                X[row_index, column_index] = random.randint(0, dof - 1)

*  0    0  100 False 5
   1  100  200 False 5
*  2  200  222 False 5
   3  222  235 True 6
   4  235  247 True 7
   5  247  258 True 8
*  6  258  358 False 5
   7  358  458 False 5
   8  458  558 False 5
   9  558  658 False 5
  10  658  758 False 5
* 11  758  800 False 5
  12  800  842 True 6
  13  842  876 True 7
  14  876  876 True 8
* 15  876  976 False 5
  16  976 1076 False 5
* 17 1076 1200 True 6
  18 1200 1200 True 7
  19 1200 1245 True 8


In [13]:
tile_bounds

[0,
 100,
 200,
 222,
 235,
 247,
 258,
 358,
 458,
 558,
 658,
 758,
 800,
 842,
 876,
 876,
 976,
 1076,
 1200,
 1200,
 1245]

In [14]:
tile_discretized

[False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True]

In [15]:
tile_group_bounds

[0, 2, 6, 11, 15, 17, 20]

In [16]:
np.savetxt("dummy_X.csv", X, delimiter="\t")

In [17]:
np.savetxt("dummy_bounds.csv", np.array(tile_bounds, dtype='int'), fmt="%i", delimiter="\t")

In [22]:
np.savetxt("dummy_groups.csv", np.array(tile_group_bounds, dtype='int'), fmt="%i", delimiter="\t")

In [18]:
np.savetxt("dummy_discretized.csv", np.array(tile_discretized, dtype='int'), fmt="%i", delimiter="\t")

In [19]:
np.savetxt("dummy_y.csv", np.random.randint(0, 3, total_row_count), fmt="%i", delimiter="\t")